# Setup

In [ ]:
# Parameters
videos_dir = None
scene_json_dirs = None
save_dir_all = None
metadata_dir_all = None
extract_metadata_only = None
# Change this to the desired number of frames per segment
num_frames_per_segment = None

In [ ]:
import os

dir_path = os.getcwd()


if not videos_dir:
    if 'google.colab' in str(get_ipython()):
        # Update this path as necessary
        videos_dir = f'{dir_path}/AIC_Video'
    elif 'kaggle' in str(get_ipython()):
        videos_dir = f'{dir_path}/AIC_Video'
    else:
        parent_dir_path = os.path.dirname(dir_path)
        videos_dir = f'{parent_dir_path}/dataset/AIC_Video'
    
if not scene_json_dirs:
    scene_json_dirs = f'{dir_path}/scene_JSON'
    
if not save_dir_all:
    save_dir_all = f'{dir_path}/keyframes'
    
if not metadata_dir_all:
    metadata_dir_all = f'{dir_path}/keyframes_metadata'
    
if not extract_metadata_only:
    extract_metadata_only = False
    
if not num_frames_per_segment:
    num_frames_per_segment = 3  # Change this to the desired number of frames per segment

In [ ]:
!git clone https://github.com/soCzech/TransNetV2.git
!git lfs install
!cd TransNetV2
!git lfs fetch https://github.com/soCzech/TransNetV2.git
!git lfs checkout

fatal: destination path 'TransNetV2' already exists and is not an empty directory.
Updated Git hooks.
Git LFS initialized.
fetch: Fetching reference refs/heads/feat/test
Checking out LFS objects: 100% (6/6), 0 B | 0 B/s, done.                        


In [ ]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm
import subprocess
import json

# Parsing data path

In [ ]:
def numpy_to_python(obj):
    """Convert numpy types to standard Python types."""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj


def parse_video_info(videos_dir):
    """Parse video information from the directory structure."""
    all_video_paths = {}
    for part in sorted(os.listdir(videos_dir)):
        data_part = part.split('_')[-1]
        all_video_paths[data_part] = {}

    for data_part in sorted(all_video_paths.keys()):
        data_part_path = os.path.join(
            videos_dir, f'Videos_{data_part}', 'video')
        video_paths = sorted(os.listdir(data_part_path))
        video_ids = [video_path.replace('.mp4', '').split(
            '_')[-1] for video_path in video_paths]
        for video_id, video_path in zip(video_ids, video_paths):
            video_path_full = os.path.join(data_part_path, video_path)
            all_video_paths[data_part][video_id] = video_path_full

    return all_video_paths

# Process cutting frames

In [ ]:
def get_evenly_spaced_frames(start_idx, end_idx, num_frames):
    """Get evenly spaced frame indices between start_idx and end_idx."""
    return np.linspace(start_idx, end_idx, num_frames, dtype=int)


def get_frame_timestamp(frame_index, fps):
    """Convert frame index to timestamp in seconds."""
    return frame_index / fps


def create_directory(directory):
    """Create a directory if it doesn't exist."""
    if not os.path.exists(directory):
        os.makedirs(directory)


def get_relative_path(path, base_dir):
    """Get the relative path from base_dir to path."""
    return os.path.relpath(path, base_dir)


def get_video_fps(video_path):
    """Get the fps of a video file."""
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    return fps

def process_metadata(video_path, scene_json_path, save_dir, videos_dir, fps, num_frames_per_segment):
    """Process and extract metadata from video segments."""
    with open(scene_json_path, 'r') as f:
        video_scenes = json.load(f)

    frame_metadata = {}
    video_id = os.path.basename(video_path).split('.')[0]

    relative_video_path = get_relative_path(video_path, videos_dir)
    keyframes_dir = os.path.dirname(os.path.dirname(save_dir))

    for i, (start, end) in enumerate(video_scenes):
        frame_indices = get_evenly_spaced_frames(
            start, end, num_frames_per_segment)

        for index in frame_indices:
            timestamp = get_frame_timestamp(index, fps)
            filename = f"{index:06d}.jpg"
            filepath = os.path.join(save_dir, filename)
            relative_frame_path = get_relative_path(filepath, keyframes_dir)

            frame_metadata[f"{video_id}_extra_{index:06d}"] = {
                "shot_index": numpy_to_python(i),
                "frame_index": numpy_to_python(index),
                "shot_start": numpy_to_python(start),
                "shot_end": numpy_to_python(end),
                "timestamp": numpy_to_python(timestamp),
                "video_path": relative_video_path,
                "frame_path": relative_frame_path
            }

    return frame_metadata


def sample_frames(video_path, scene_json_path, save_dir, videos_dir, fps, num_frames_per_segment, width=None, height=None):
    """Sample frames from video segments, resize them, and save them to the specified directory."""
    create_directory(save_dir)

    frame_metadata = process_metadata(
        video_path, scene_json_path, save_dir, videos_dir, fps, num_frames_per_segment)

    cap = cv2.VideoCapture(video_path)

    for frame_id, metadata in frame_metadata.items():
        index = metadata['frame_index']
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)
        ret, frame = cap.read()
        if ret:
            # Resize the frame if width and height are provided
            if width is not None and height is not None:
                frame = cv2.resize(frame, (width, height))
            
            filename = f"{index:06d}.jpg"
            filepath = os.path.join(save_dir, filename)
            if not cv2.imwrite(filepath, frame):
                print('Failed to save frame:', filepath)
        else:
            print('Failed to read frame at index:', index)
    
    cap.release()
    return frame_metadata

def process_videos(all_video_paths, scene_json_dirs, save_dir_all, metadata_dir_all, videos_dir, num_frames_per_segment, extract_only=False, width= 640, height= 640):
    """Process all videos: sample frames (if needed) and extract metadata."""
    create_directory(save_dir_all)
    create_directory(metadata_dir_all)

    for key in all_video_paths.keys():
        # save_dir = os.path.join(save_dir_all, key)

        # if not all_video_paths[key]:
        #     print(f"Skipping empty AIC_Video subdirectory: {key}")
        #     continue

        scene_json_subdir = os.path.join(scene_json_dirs, key)
        if not os.path.exists(scene_json_subdir) or not os.listdir(scene_json_subdir):
            print(
                f"Skipping empty or non-existent Scene_JSON subdirectory: {key}")
            continue

        # create_directory(save_dir)

        video_paths_dict = all_video_paths[key]
        video_ids = sorted(video_paths_dict.keys())

        # Calculate fps once for this group of videos
        first_video_path = video_paths_dict[video_ids[0]]
        fps = get_video_fps(first_video_path)

        for video_id in tqdm(video_ids, desc=f"Processing videos in {key}"):
            video_path = video_paths_dict[video_id]
            video_scene_path = os.path.join(
                scene_json_dirs, key, f"{video_id}.json")

            if not os.path.exists(video_scene_path):
                print(f"Skipping missing Scene_JSON file: {video_scene_path}")
                continue

            save_dir_video = os.path.join(save_dir_all, f"{key}_{video_id}_extra")

            if extract_only:
                frame_metadata = process_metadata(
                    video_path, video_scene_path, save_dir_video, videos_dir, fps, num_frames_per_segment)
            else:
                frame_metadata = sample_frames(
                    video_path, video_scene_path, save_dir_video, videos_dir, fps, num_frames_per_segment, width= width, height= height)

            # Save metadata for this video
            metadata_filename = os.path.join(
                metadata_dir_all, f"{key}_{video_id}_extra.json")
            with open(metadata_filename, 'w') as f:
                json.dump(frame_metadata, f, default=numpy_to_python)

In [ ]:
all_video_paths = parse_video_info(videos_dir)
process_videos(all_video_paths, scene_json_dirs, save_dir_all,
               metadata_dir_all, videos_dir, num_frames_per_segment, extract_only=False, width= 640, height= 640)

Processing videos in L01:   0%|          | 0/2 [00:00<?, ?it/s][h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref short failure
[h264 @ 0x5ef7d312db00] mmco: unref s

# Create metadata keyframe for organizing committee (option)

In [ ]:
import pandas as pd
import json

In [ ]:
keyframes_path = f'{save_dir_all}'
organizing_committee_name_dir = os.listdir(keyframes_path)
organizing_committee_keyframe_paths = [f"{keyframes_path}/{name}" for name in organizing_committee_name_dir if not name.endswith('extra')]

for organizing_committee_keyframe_path in organizing_committee_keyframe_paths:
    
    video_path = f"{videos_dir}/Videos_{organizing_committee_keyframe_path.split('/')[-1].split('_')[0]}/video/{organizing_committee_keyframe_path.split('/')[-1]}.mp4"
    scene_json_path = f"{scene_json_dirs}/{organizing_committee_keyframe_path.split('/')[-1].split('_')[0]}/{organizing_committee_keyframe_path.split('/')[-1].split('_')[1]}.json"
    csv_file_path = f"{dir_path}/map-keyframes/{organizing_committee_keyframe_path.split('/')[-1]}.csv"
    df = pd.read_csv(csv_file_path)
    
    with open(scene_json_path, 'r') as f:
        video_scenes = json.load(f)
    
    json_data = {}
    # Iterate through each row in the dataframe to populate the JSON structure
    for index, row in df.iterrows():
        frame_index = int(row['frame_idx'])  # Assuming your CSV has a column 'frame_index'
        
        for idx, (start, end) in enumerate(video_scenes):
            if (frame_index >= start) and (frame_index <= end):
                key = f"{organizing_committee_keyframe_path.split('/')[-1]}_{frame_index:06d}"  # Create the key with zero-padded frame index
                json_data[key] = {
                    "shot_index": idx,  # Based on its index in scene_JSON
                    "frame_index": frame_index,
                    "shot_start": start,  # Based on its start of shot in range of scene_JSON
                    "shot_end": end,     # Based on its end of shot in range of scene_JSON
                    "timestamp": float(row['pts_time']),
                    "video_path": get_relative_path(video_path, videos_dir),
                    "frame_path": get_relative_path(f"{organizing_committee_keyframe_path}/{int(row['n']):03d}.jpg", os.path.dirname(keyframes_path))
                }

                # Convert the dictionary to a JSON string
                json_output = json.dumps(json_data)

                # Save the JSON to a file
                json_output_path = f"{dir_path}/keyframes_metadata/{organizing_committee_keyframe_path.split('/')[-1]}.json"
                with open(json_output_path, 'w') as json_file:
                    json_file.write(json_output)

# Rename following structure

In [ ]:
import json
import re

In [ ]:
def rename_keys(json_file):
    # Read the JSON file
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # New dictionary to store renamed keys
    new_data = {}
    
    # Regular expression to match the key pattern
    pattern = r'(L\d+_V\d+)_(\d+)_(extra)'
    
    # Rename keys
    for key, value in data.items():
        if re.match(pattern, key):
            new_key = re.sub(pattern, r'\1_\3_\2', key)
            new_data[new_key] = value
        else:
            new_data[key] = value  # Keep keys that don't match the pattern unchanged
    
    # Write the updated data back to the JSON file
    with open(json_file, 'w') as f:
        json.dump(new_data, f, indent=2)
        
        
def process_folder(folder_path):
    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('_extra.json'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {filename}")
            rename_keys(file_path)
            print(f"Completed processing: {filename}")

In [ ]:
process_folder(metadata_dir_all)
print("Keys renamed successfully.")

Processing file: L01_V001_extra.json
Completed processing: L01_V001_extra.json
Processing file: L01_V002_extra.json
Completed processing: L01_V002_extra.json
Keys renamed successfully.
